In [2]:
%run ../../../main.py
%matplotlib inline

In [7]:
import pandas as pd

from pyarc.algorithms import M1Algorithm, M2Algorithm, top_rules, createCARs 
from pyarc.data_structures import TransactionDB


import pyarc.qcba as qcba
from pyarc import CBA
from pyarc.qcba.data_structures import *
import matplotlib.pyplot as plt

from pyarc.qcba import QCBA

import pandas as pd
from pyarc.qcba.data_structures import (
    IntervalReader,
    Interval,
    QuantitativeDataFrame,
    QuantitativeCAR
)

interval_reader = IntervalReader()

interval_reader.closed_bracket = "", "NULL"
interval_reader.open_bracket = "NULL", ""
interval_reader.infinity_symbol = "inf", "inf"
interval_reader.members_separator = "_to_"

interval_reader.compile_reader()

i = interval_reader.read("82.9815_to_inf")

QuantitativeCAR.interval_reader = interval_reader

In [8]:
#
#
# =========================
# Oveření běhu v závislosti na vložených pravidlech / instancích
# =========================
#
#
#

import time

rule_count = 100

benchmark_data = {
    "input rows": [],
    "input rules": [],
    "output rules M1 pyARC": [],
    "output rules M1 pyARC unique": [],
    "output rules M2 pyARC": [],
    "output rules pyARC qCBA": [],
    "time M1 pyARC": [],
    "time M1 pyARC unique": [],
    "time M2 pyARC": [],
    "time pyARC qCBA": []
}

stop_m2 = True

number_of_iterations = 10

directory = "c:/code/python/machine_learning/assoc_rules"

dataset_name_benchmark = "lymph0"

pd_ds = pd.read_csv("c:/code/python/machine_learning/assoc_rules/train/{}.csv".format(dataset_name_benchmark))
pd_ds_quant = pd.read_csv("c:/code/python/machine_learning/assoc_rules/folds_undiscr/train/{}.csv".format(dataset_name_benchmark))


for i in range(5):
    dataset_name_benchmark = "lymph0"
    
    pd_ds = pd.concat([pd_ds, pd_ds])
    pd_ds_quant = pd.concat([pd_ds_quant, pd_ds_quant])
    quant_dataframe = QuantitativeDataFrame(pd_ds_quant)
    
    txns = TransactionDB.from_DataFrame(pd_ds, unique_transactions=True)
    txns_unique = TransactionDB.from_DataFrame(pd_ds, unique_transactions=False) 
    
    rules = top_rules(txns.string_representation, appearance=txns.appeardict, target_rule_count=rule_count)

    cars = createCARs(rules)
     
    if len(cars) > rule_count:
        cars = cars[:rule_count]    

        
    m1t1 = time.time()
    m1clf_len = []
    for _ in range(number_of_iterations):
        m1 = M1Algorithm(cars, txns)
        clf = m1.build()
        m1clf_len.append(len(clf.rules) + 1)
    
    m1t2 = time.time()
    
    
    t1_qcba = time.time()
    qcba_clf_len = []
    for _ in range(number_of_iterations):
        m1 = M1Algorithm(cars, txns)
        clf = m1.build()
        m1clf_len.append(len(clf.rules) + 1)
        
        rm_cba = CBA()
        rm_cba.clf = clf
        
        rm_qcba = QCBA(rm_cba, quant_dataframe)
        rm_qcba.fit()
    
    
    t2_qcba = time.time()
    
    
    m1t1_unique = time.time()
    m1clf_len_unique = []
    for _ in range(number_of_iterations):
        m1 = M1Algorithm(cars, txns_unique)
        clf = m1.build()
        m1clf_len_unique.append(len(clf.rules) + 1)
    
    m1t2_unique = time.time()
    
    
    
    if not stop_m2:
        m2t1 = time.time()
        m2clf_len = []
        for _ in range(number_of_iterations):
            m2 = M2Algorithm(cars, txns)
            clf = m2.build()
            m2clf_len.append(len(clf.rules) + 1)

        m2t2 = time.time()
    
     
    m1duration = (m1t2 - m1t1) / number_of_iterations
    m1duration_unique = (m1t2_unique - m1t1_unique) / number_of_iterations
    outputrules_m1 = sum(m1clf_len) / len(m1clf_len)
    outputrules_m1_unique = sum(m1clf_len_unique) / len(m1clf_len_unique)
    
    if not stop_m2:
        m2duration = (m2t2 - m2t1) / number_of_iterations
        outputrules_m2 = sum(m2clf_len) / len(m2clf_len)
        if m2duration > 0.5:
            stop_m2 = True
    
    benchmark_data["input rows"].append(len(txns))
    benchmark_data["input rules"].append(rule_count)
    benchmark_data["output rules M1 pyARC"].append(outputrules_m1)
    benchmark_data["output rules M1 pyARC unique"].append(outputrules_m1_unique)
    benchmark_data["output rules M2 pyARC"].append(None if stop_m2 else outputrules_m2)
    benchmark_data["time M1 pyARC"].append(m1duration)
    benchmark_data["time M1 pyARC unique"].append(m1duration_unique)
    benchmark_data["time M2 pyARC"].append(None if stop_m2 else m2duration)

    print("data_count:", len(txns))
    print("M1 duration:", m1duration)
    print("M1 unique duration", m1duration_unique)
    print("M1 output rules", outputrules_m1)
    if not stop_m2:
        print("M2 duration:", m2duration)
        print("M2 output rules", outputrules_m2)
    print("\n\n")

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=19
Rule count: 2266, Iteration: 1
Target rule count satisfied: 100
applying selected transformations
refitting
literal pruning
trimming
extending
[                                                  ]


KeyboardInterrupt: 